In [1]:
import random
from pprint import pprint
import io
import imageio.v2 as imageio
import matplotlib.pyplot as plt
import numpy as np
import time
import logging

import tyssue
from tyssue import History
from tyssue.dynamics import effectors, model_factory, model_factory_vessel, model_factory_cylinder
from tyssue.generation.hexagonal_grids import hexa_cylinder
from tyssue.generation.shapes import sheet_from_cell_centers
from tyssue.geometry.sheet_geometry import CylinderGeometry, CylinderGeometryInit
from tyssue import config
from tyssue.draw import sheet_view, create_gif
from tyssue.geometry.vessel_geometry import VesselGeometry as geom
from tyssue.solvers.viscous import EulerSolver
from tyssue.draw.ipv_draw import browse_history
from tyssue.topology.sheet_topology import cell_division, drop_face, split_vert
from tyssue.topology.base_topology import remove_face
from tyssue.behaviors import EventManager
from tyssue.generation.modifiers import create_anchors
try:
    from ..behaviors.behaviors import *
except:
    print("Failed relative import, attempting direct import")
    from vivarium_tyssue.behaviors import *
import ipyvolume as ipv

from process_bigraph import Process, Composite, ProcessTypes
from process_bigraph.emitter import emitter_from_wires, gather_emitter_results

%load_ext snakeviz

C++ extensions are not available for this version
collision solver could not be imported You may need to install CGAL and re-install tyssue
Failed relative import, attempting direct import


In [2]:
# def fix_points_cylinder(sheet, radius):
#     xy = sheet.vert_df[['x', 'y']].to_numpy()
#     r = np.linalg.norm(xy, axis=1)
#     r_safe = np.where(r == 0, 1e-12, r)
#     xy_on_cylinder = (radius/r_safe)[:, None] * xy
#     sheet.vert_df['x'] = xy_on_cylinder[:, 0]
#     sheet.vert_df['y'] = xy_on_cylinder[:, 1]

In [3]:
# def divide_cylinder(sheet, manager, geom, division_rate, dt, radius):
#     update_stem_cells(sheet)
#     stem_cells = sheet.face_df.loc[(sheet.face_df["stem_cell"] == 1) & (sheet.face_df["area"]>=0.7*sheet.face_df["area"].mean())].copy()
#     n_stem = len(stem_cells)
#     cell_ids = list(stem_cells["unique_id"])
#     n_divisions = np.random.binomial(n=n_stem, p=division_rate * dt)
#     dividing_cells = np.random.choice(cell_ids, size=n_divisions, replace=False)
#
#     for cell_id in dividing_cells:
#         cell_idx = int(sheet.face_df[sheet.face_df["unique_id"] == cell_id].index[0])
#         daughter = cell_division(sheet, cell_idx, geom)
#     manager.append(divide_cylinder, geom=geom, division_rate=division_rate, dt=dt, radius=radius)
#     fix_points_cylinder(sheet, radius=radius)

In [4]:
# def apoptosis_cylinder(sheet, manager, death_rate, dt, radius, geom):
#     update_stem_cells(sheet)
#     dying_cells = sheet.face_df.loc[sheet.face_df["dying_cell"] == 1]
#     n_dying = len(dying_cells)
#     cell_ids = list(dying_cells["unique_id"])
#     n_deaths = np.random.binomial(n=n_dying, p=death_rate * dt)
#     to_kill = np.random.choice(cell_ids, size=n_deaths, replace=False)
#
#     for cell_id in to_kill:
#         cell_idx = int(sheet.face_df[sheet.face_df["unique_id"] == cell_id].index[0])
#         # if sheet.face_df.loc[cell_idx, "boundary"] == 0:
#         vertex = remove_face(sheet, cell_idx)
#         # else:
#         #     vertex = drop_face(sheet, cell_idx, geom)
#         # split_vert(sheet, vertex)
#     manager.append(apoptosis_cylinder, death_rate=death_rate, dt=dt, radius=radius, geom=geom)
#     fix_points_cylinder(sheet, radius=radius)

In [5]:
def auto_correlated_tension(sheet, manager, sigma, phi):
    """
    Parameters:
    sheet: tyssue.Sheet object
    manager: EventManager object
    sigma: float, standard deviation
    phi: float, autocorrelation factor between 0 and 1 (AR(1) autocorrelation factor)
        closer to 1 means stronger correlation
    Returns:
    updates sheet.edge_df["line_tension"]
    """
    noise = np.random.normal(loc=0, scale=sigma, size=len(sheet.edge_df["line_tension"]))
    sheet.edge_df["line_tension"] =  phi * sheet.edge_df["line_tension"] + noise
    manager.append(auto_correlated_tension, sigma=sigma, phi=phi)

In [6]:
# def update_stem_cells(eptm):
#     eptm.face_df['stem_cell'] = 0
#     eptm.face_df['dying_cell'] = 0
#     eptm.face_df.loc[(eptm.face_df["boundary"] != 1) & (eptm.face_df["z"] < 0), "stem_cell"] = 1
#     eptm.face_df.loc[(eptm.face_df["z"] > 0), "dying_cell"] = 1

In [7]:
radius = 2.576
#generate cylindrical tissue
points_xyz = hexa_cylinder(16,30, radius = radius, noise = 0.0, capped = True)
sheet = sheet_from_cell_centers(points_xyz)
sheet = sheet.extract_bounding_box(z_boundary = (-10.1, 30), coords=['x', 'y', 'z'])
sheet.sanitize(trim_borders=False)
create_anchors(sheet)
sheet = sheet.extract_bounding_box(z_boundary = (-10.1, 10), coords=['x', 'y', 'z'])
sheet.sanitize(trim_borders=False)
geom.update_all(sheet)
sheet.network_changed = False

/Users/tasnifrahman/PycharmProjects/vivatyssue/src/tyssue/generation/modifiers.py:193: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sheet.edge_df = pd.concat([sheet.edge_df, anchor_edge_df], sort=True)


In [8]:
sheet.face_df.loc[(sheet.face_df["z"]<8) & (sheet.face_df["z"]>-8), "area"].mean()

np.float64(1.0001577509936472)

In [9]:
draw_specs = tyssue.config.draw.sheet_spec()
draw_specs["face"]["visible"] = True
draw_specs["face"]["visible"] = True
draw_specs["face"]["alpha"] = 0.2
ipv.clear()
fig, mesh = sheet_view(sheet, coords=['x', 'y', 'z'], mode="3D", **draw_specs)
ipv.pylab.xyzlabel("x-axis", "y-axis", "z-axis")
ipv.pylab.style.axes_on()
fig

Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=45.0, position=(0.0,…

In [10]:
#generate model
model = model_factory_vessel([
    effectors.LineTension,
    effectors.FaceAreaElasticity,
    effectors.PerimeterElasticity,
], effectors.FaceAreaElasticity)

In [11]:
#set model parameters
sheet.face_df["area_elasticity"] = 1
sheet.face_df["prefered_area"] = 1
sheet.face_df["perimeter_elasticity"] = 1
sheet.face_df["prefered_perimeter"] = 3.5
sheet.edge_df["line_tension"] = np.random.normal(loc=0, scale=0.01, size=len(sheet.edge_df))
sheet.edge_df["is_active"] = 1
sheet.vert_df["viscosity"] = 5
sheet.vert_df["prefered_deviation"] = 0
sheet.vert_df["surface_elasticity"] = 5
sheet.settings["threshold_length"] = 0.1
geom.update_all(sheet)

In [12]:
sheet.edge_df["face"]

edge
0       123
1       123
2       123
3       123
4       123
       ... 
1915    284
1916    284
1917    284
1918    284
1919     68
Name: face, Length: 1920, dtype: int64

In [13]:
history = History(sheet)

In [14]:
#set time-step
dt=0.1

In [15]:
#initialize manager and add division and apoptosis
#set division and apoptosis parameters
manager = EventManager()
manager.append(divide_cylinder, geom=geom, division_rate=0.0005, dt=dt, radius=radius)
manager.append(apoptosis_cylinder, death_rate=0.0005, dt=dt, radius=radius, geom=geom)
manager.append(auto_correlated_tension, sigma = 0.01, phi=0.5)

In [16]:
#initialize solver
solver = EulerSolver(
    sheet,
    geom,
    model,
    history=history,
    auto_reconnect=True,
    manager=manager,
)

In [17]:
sheet.edge_df["is_active"]

edge
0       1
1       1
2       1
3       1
4       1
       ..
1915    1
1916    1
1917    1
1918    1
1919    1
Name: is_active, Length: 1920, dtype: int64

In [18]:
#set time variables
dt = dt
tf = 2000
sheet.settings["dt"] = dt
sheet.settings["p_4"] = 1/(dt)
sheet.settings["p_5p"] = 1/(dt)
#run simulation
start = time.time()
res = solver.solve(tf=tf, dt = dt)
print(f"Time Elapsed: {time.time()-start}")

Time Elapsed: 1017.9212019443512


In [19]:
sheet.edge_df

,at_border,boundary,cell,cell_o,cx,cy,cz,dx,dy,dz,...,trgt,trgt_o,tx,ty,tz,unique_id,unique_id_max,ux,uy,uz
edge,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0.0,0.0,0.0,0.003166,0.334110,-0.009436,...,114,844,-2.568879,0.191409,-7.694431,10,6120,0.009471,0.999571,-0.028230
1,0,0,0,0,0.0,0.0,0.0,-0.075826,0.244993,0.017414,...,132,842,-2.495693,-0.638194,-7.071934,12,6120,-0.294966,0.953032,0.067741
2,0,0,0,0,0.0,0.0,0.0,-0.082673,0.200003,1.082262,...,131,841,-2.419867,-0.883187,-7.089348,14,6120,-0.074906,0.181213,0.980585
3,0,0,0,0,0.0,0.0,0.0,0.181035,-0.540698,0.234869,...,130,840,-2.337194,-1.083190,-8.171610,16,6120,0.293545,-0.876730,0.380835
4,0,0,0,0,0.0,0.0,0.0,0.013500,-0.538983,0.051089,...,495,839,-2.554302,-0.333639,-8.840515,18,6120,0.024927,-0.995200,0.094332
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1706,0,0,0,0,0.0,0.0,0.0,-0.359772,0.040053,0.270319,...,595,725,-0.464202,-2.533830,-5.017863,2060,6120,-0.796334,0.088654,0.598336
1707,0,0,0,0,0.0,0.0,0.0,-0.407605,0.045432,0.325829,...,521,725,-0.488253,-2.529305,-3.678541,2060,6120,-0.778040,0.086721,0.621945
1708,0,0,0,0,0.0,0.0,0.0,0.407605,-0.045432,-0.325829,...,596,728,-0.080648,-2.574737,-4.004370,1969,6120,0.778040,-0.086721,-0.621945


In [20]:
ipv.clear()

In [21]:
ipv.clear()
browse_history(history,**draw_specs)

Container(figure=Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=45.…

interactive(children=(IntSlider(value=0, description='i', max=20000), Output()), _dom_classes=('widget-interac…

In [22]:
len(sheet.face_df)

288

In [23]:
sheet.edge_df["is_active"]

edge
0       1
1       1
2       1
3       1
4       1
       ..
1706    1
1707    1
1708    1
1709    1
1710    1
Name: is_active, Length: 1711, dtype: int64

In [24]:
ipv.clear()
fig, mesh = sheet_view(sheet, coords=['x', 'y', 'z'], mode="3D", **draw_specs)
ipv.pylab.xyzlabel("x-axis", "y-axis", "z-axis")
ipv.pylab.style.axes_on()
fig

Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=45.0, position=(0.0,…

In [25]:
sheet1 = history.retrieve(0.0)

In [26]:
sheet.edge_df

,at_border,boundary,cell,cell_o,cx,cy,cz,dx,dy,dz,...,trgt,trgt_o,tx,ty,tz,unique_id,unique_id_max,ux,uy,uz
edge,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0.0,0.0,0.0,0.003166,0.334110,-0.009436,...,114,844,-2.568879,0.191409,-7.694431,10,6120,0.009471,0.999571,-0.028230
1,0,0,0,0,0.0,0.0,0.0,-0.075826,0.244993,0.017414,...,132,842,-2.495693,-0.638194,-7.071934,12,6120,-0.294966,0.953032,0.067741
2,0,0,0,0,0.0,0.0,0.0,-0.082673,0.200003,1.082262,...,131,841,-2.419867,-0.883187,-7.089348,14,6120,-0.074906,0.181213,0.980585
3,0,0,0,0,0.0,0.0,0.0,0.181035,-0.540698,0.234869,...,130,840,-2.337194,-1.083190,-8.171610,16,6120,0.293545,-0.876730,0.380835
4,0,0,0,0,0.0,0.0,0.0,0.013500,-0.538983,0.051089,...,495,839,-2.554302,-0.333639,-8.840515,18,6120,0.024927,-0.995200,0.094332
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1706,0,0,0,0,0.0,0.0,0.0,-0.359772,0.040053,0.270319,...,595,725,-0.464202,-2.533830,-5.017863,2060,6120,-0.796334,0.088654,0.598336
1707,0,0,0,0,0.0,0.0,0.0,-0.407605,0.045432,0.325829,...,521,725,-0.488253,-2.529305,-3.678541,2060,6120,-0.778040,0.086721,0.621945
1708,0,0,0,0,0.0,0.0,0.0,0.407605,-0.045432,-0.325829,...,596,728,-0.080648,-2.574737,-4.004370,1969,6120,0.778040,-0.086721,-0.621945


In [27]:
ipv.clear()
browse_history(history, **draw_specs)

Container(figure=Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=45.…

interactive(children=(IntSlider(value=0, description='i', max=20000), Output()), _dom_classes=('widget-interac…

In [28]:
sheet.vert_df

,x,y,z,is_active,rho,height,basal_shift,unique_id,unique_id_max,srce_o,boundary,at_border,is_anchor,tx,ty,tz,viscosity,prefered_deviation,surface_elasticity
vert,,,,,,,,,,,,,,,,,,,
0,0.772309,2.457502,-3.366893,1,0.395461,0.0,4.0,16,1020,18,0,0,0,0.953999,-0.299809,0.0,5,0,5
1,1.644295,1.982944,-3.404615,1,0.395461,0.0,4.0,17,1020,19,0,0,0,0.769777,-0.638313,0.0,5,0,5
2,-0.348424,-2.552328,8.071026,1,0.395913,0.0,4.0,77,1020,83,0,0,0,-0.990810,0.135258,0.0,5,0,5
3,0.230328,-2.565682,9.522364,1,0.394964,0.0,4.0,79,1020,85,0,0,0,-0.995995,-0.089413,0.0,5,0,5
4,1.636775,-1.989157,8.145520,1,0.395913,0.0,4.0,80,1020,86,0,0,0,-0.772188,-0.635394,0.0,5,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592,1.250583,2.252070,-4.730610,1,0.395132,0.0,4.0,581,1020,628,0,0,0,0.874251,-0.485475,0.0,5,0,5
593,2.104353,1.485757,-7.424072,1,0.394964,0.0,4.0,845,1020,1003,0,0,0,0.576769,-0.816907,0.0,5,0,5
594,2.462536,0.756104,-6.518248,1,0.395132,0.0,4.0,521,1020,558,0,0,0,0.293519,-0.955953,0.0,5,0,5


In [30]:
draw_specs

{'edge': {'visible': True,
  'width': 0.5,
  'head_width': 0.0,
  'length_includes_head': True,
  'shape': 'right',
  'color': '#2b5d0a',
  'alpha': 0.8,
  'zorder': 1,
  'colormap': 'viridis'},
 'vert': {'visible': False,
  's': 100,
  'color': '#000a4b',
  'alpha': 0.3,
  'zorder': 2},
 'grad': {'color': '#000a4b', 'alpha': 0.5, 'width': 0.04},
 'face': {'visible': True,
  'color': '#8aa678',
  'alpha': 0.2,
  'zorder': -1,
  'visibles': True},
 'axis': {'autoscale': True,
  'color_bar': False,
  'color_bar_cmap': 'viridis',
  'color_bar_range': False,
  'color_bar_label': False,
  'color_bar_target': 'face'}}